# STA 141B Assignment 2

Due __Februrary 9, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10. 

__Exercise 1__

We will use the [lichess](https://lichess.org/api) API to retrieve some information about the current state of chess in the world. In order to answer below questions, make precise and economical requests. You may use:
```
import requests
import json
import pandas

from datetime import datetime
```

In [97]:
import requests
import json
import pandas
from datetime import datetime

__(a)__ What is the real name of the player `muisback`? 

In [7]:
###Access API Token
def read_key(keyfile):
    with open(keyfile) as f:
        return f.readline().strip("\n")
key = read_key("../hw2/lichessAPItoken.txt")

In [3]:
response = requests.get("https://lichess.org/api/user/muisback", params={"profile":True}, headers= {
    'Authorization' : f'Bearer {key}',
})
account_data = response.json()
full_name = f'{account_data["profile"]["firstName"]}, {account_data["profile"]["lastName"]}'
full_name

'Rauf, Mamedov'

__(b, i)__ Get the username of the last player that played a rapid game against user `athena-pallada` in 2023. __(ii)__ In all games against this user, what is the win-to-loss ratio of `athena-pallada`?

In [185]:
response = requests.get("https://lichess.org/api/games/user/athena-pallada", 
    params={'perfType' : "rapid",'max' : 5, 'until' : 1704009600000}, #1704009600000 is 2023 in milliseconds since epoch
    headers= {'Authorization' : f'Bearer {key}'})
for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                # Process the chunk as needed (e.g., save to a file)
                print(chunk.decode('utf-8'), end='')

[Event "GM athena-pallada simul"]
[Site "https://lichess.org/FWHyjmqs"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "Bacio129"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1500"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A40"]
[Termination "Time forfeit"]

1. d4 1-0


[Event "GM athena-pallada simul"]
[Site "https://lichess.org/fEu5PbLv"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "lawrencepl"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1584"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A00"]
[Termination "Normal"]

1. b4 Nf6 1-0


[Event "GM athena-pallada simul"]
[Site "https://lichess.org/0Bo3Zbbd"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "wranglerek"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1665"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "C42"]
[T

In [186]:
response = requests.get("https://lichess.org/api/games/user/athena-pallada", 
    params = {
        'vs' : 'Bacio129'
    },
    headers = {
        'Authorization' : f'Bearer {key}'
    })
for chunk in response.iter_content(chunk_size=1024): #because is not returned as json, iter through response
            if chunk:
                print(chunk.decode('utf-8'), end='')

[Event "GM athena-pallada simul"]
[Site "https://lichess.org/FWHyjmqs"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "Bacio129"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1500"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A40"]
[Termination "Time forfeit"]

1. d4 1-0


[Event "GM athena-pallada simul"]
[Site "https://lichess.org/7BzFYJsB"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "Bacio129"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:15:14"]
[WhiteElo "2837"]
[BlackElo "1500"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A01"]
[Termination "Time forfeit"]

1. b3 d5 2. Bb2 Nc6 3. e3 e5 4. Bb5 Bd6 5. f4 Bd7 6. Nf3 Nge7 7. Bxc6 Nxc6 8. fxe5 Be7 9. O-O O-O 10. Nc3 Bg4 11. d4 Qd7 12. h3 Bh5 13. Ne2 f6 14. Nf4 Bf7 15. Qe2 fxe5 16. Nxe5 Nxe5 17. dxe5 Be6 18. Rad1 Rad8 19. c4 c6 20. Kh2 Bc5 21. cxd5 cxd5 22. Bd4 Bxd4 23. exd4 Qc6 24. Rc1 Qd7 25. Nxe6 Qxe6 26. Rxf8+ Rxf8 27. Rc7 

The user name of the last player athena-pallada has played against is Bacio129. In all the games against this player, which is two games, Athena-Pallada has won both games, so they have a win ratio of 2:0. 

__(c)__ Consider the top ten players in the bullet leaderboard. __(i)__ Which player has the most bullet games overall? __(ii)__ Which player has played the most bullet games relative to account age in days? __(iii)__ Which player has the worst win-to-loss ratio over all formats?

In [68]:
response = requests.get('https://lichess.org/api/player/top/10/bullet', 
    params = {
        
        }, 
    headers = {
        'Authorization' : f'Bearer {key}'
    })
top_bullet_players = response.json()

In [69]:
bullet_player_user = []
for i in range(10):
    bullet_player_user.append(top_bullet_players['users'][i]['username'])
bullet_player_user

['RedWarrior25',
 'Ediz_Gurel',
 'anhghOst24',
 'Yulkaaa',
 'Zhigalko_Sergei',
 'V_M',
 'chessmaster2006',
 'Italianchessstar',
 'HowardXue',
 'aaryan_varshney']

In [70]:
total_bullet_games = []
for player in bullet_player_user:
    link = f'https://lichess.org/api/user/{player}/perf/bullet'
    indiv_player = requests.get(link, 
        params = {
            
        },
        headers = {
            'Authorization' : f'Bearer {key}'
        })
    total_bullet_games.append([player, indiv_player.json()['stat']['count']['all']])
total_bullet_games


[['RedWarrior25', 761],
 ['Ediz_Gurel', 5507],
 ['anhghOst24', 3428],
 ['Yulkaaa', 785],
 ['Zhigalko_Sergei', 74408],
 ['V_M', 2535],
 ['chessmaster2006', 6674],
 ['Italianchessstar', 2797],
 ['HowardXue', 5418],
 ['aaryan_varshney', 7801]]

In [71]:
max_player = max(total_bullet_games, key=lambda x: x[1])
max_player

['Zhigalko_Sergei', 74408]

In [72]:
player_counter = 0
for player in bullet_player_user:
    link = f'https://lichess.org/api/user/{player}'
    indiv_player = requests.get(link, 
        params = {
            
        },
        headers = {
            'Authorization' : f'Bearer {key}'
        })
    account_creation = indiv_player.json()["createdAt"]
    date_created = datetime.utcfromtimestamp(account_creation / 1000.0)
    curr_date = datetime.strptime('2024-02-05', '%Y-%m-%d')
    days_since = (curr_date - date_created).days
    total_bullet_games[player_counter].append(days_since)
    total_wins = indiv_player.json()["count"]["win"]
    total_loss = indiv_player.json()["count"]["loss"]
    total_bullet_games[player_counter].append(total_wins/total_loss)
    player_counter += 1
    

In [73]:
total_bullet_games

[['RedWarrior25', 761, 1709, 4.2],
 ['Ediz_Gurel', 5507, 326, 2.509646302250804],
 ['anhghOst24', 3428, 683, 2.428735302552337],
 ['Yulkaaa', 785, 394, 2.1130434782608694],
 ['Zhigalko_Sergei', 74408, 1963, 3.883759866060751],
 ['V_M', 2535, 2861, 1.9012760241773001],
 ['chessmaster2006', 6674, 2468, 1.9985632183908046],
 ['Italianchessstar', 2797, 752, 2.0332681017612524],
 ['HowardXue', 5418, 2206, 1.790632318501171],
 ['aaryan_varshney', 7801, 1750, 2.4479838709677417]]

In [74]:
bullet_games_to_days = max(total_bullet_games, key=lambda x: x[1]/x[2])
bullet_games_to_days

['Zhigalko_Sergei', 74408, 1963, 3.883759866060751]

In [77]:
worst_winloss_ratio = min(total_bullet_games, key=lambda x: x[3])
worst_winloss_ratio

['HowardXue', 5418, 2206, 1.790632318501171]

i) Considering the top 10 players in bullet, the player Zhigalko_Sergei has the most bullet games with 74408 bullet games
ii) Similarly, Zhigalko_Sergei has the most bullet games played relative to account age in days with 74408 games played in an account age of 1963 days.
iii) Across the top 10 players in bullet the played HowardXue has the worst win loss ratio with a win loss ratio of 1.7906.

__(d)__ Get all games from user `manwithavan`. Group them by opening and print the ten most popular. 

In [108]:

link = f'https://lichess.org/api/games/user/manwithavan'
indiv_player = requests.get(link, 
    params = {
        "opening" : True,
    },
    headers = {
        'Authorization' : f'Bearer {key}'
    })


In [182]:
openings = []
player_text = indiv_player.text.split('\n')
for player in player_text:
    if player[:8] == '[Opening':
        player = player.strip('[]')[8:]
        player = player.strip('""')
        openings.append(player)

In [183]:
import pandas as pd
num_opening = {}
for opening in openings:
    if opening in num_opening:
        num_opening[opening] += 1
    else:
        num_opening[opening] = 1
openings = pd.DataFrame(num_opening.items(), columns=['Opening', 'Occurences'])

In [184]:
openings = openings.sort_values(by="Occurences",ascending=False)
openings = openings.reset_index(drop=True)
openings.head(10)

,Opening,Occurences
0,Van't Kruijs Opening,7
1,Nimzo-Larsen Attack: Modern Variation,7
2,Mieses Opening,6
3,Pirc Defense,6
4,Zukertort Opening: Kingside Fianchetto,5
5,Caro-Kann Defense: Breyer Variation,5
6,?,5
7,Nimzo-Larsen Attack,5
8,Zukertort Opening: Queenside Fianchetto Variation,5
9,Queen's Pawn Game,5


__Exercise 2__

As a public organization, the compensations of employees of all institutions of the University of California are freely accessible. These reports cover UC's career faculty and staff employees, as well as part-time, temporary and student employees. See [here](https://ucannualwage.ucop.edu). Internally, the data requested by the search mask is queried using an undocumented API. For this exercise, you may use: 
```
import requests
import pandas

from json import loads
```

_Hint: If you encounter an error when parsing the data, try to use string methods (e.g., `str.replace`) to deal with them._

__(a)__ Get the compensation information of all UC Davis employees that received a gross pay that exceeded 300000 USD per year for the years 2019 to 2020. Sort the resulting table by year and last name, and print the first six entries.

In [231]:
table.head(2)

,year,location,firstname,lastname,title,gross,regular,overtime,other
0,2019,Davis,LEONARD,ABBEDUTO,PROF-HCOMP,374157.0,315450.0,0.0,58707.0
1,2019,Davis,MEHRDAD,ABEDI,PROF OF CLIN-HCOMP,310945.0,173831.0,0.0,137114.0


 __(b)__ Report the mean compensation for each title type: For gross pay, other pay and overtime pay, report the top six titles together with the number of counts. 

In [254]:
grouped_table.head(2)

,gross,regular,overtime,other,count
title,,,,,
VC AND DEAN SOM,960075.0,759375.0,0.0,200700.0,1
COO MED CTR,877439.5,622980.5,0.0,254459.0,2
